<!-- ![](./header.png) -->
<img src="./header.png",width=100%>

# The DC2 Cookbook: Recipes for Emulating the LSST DM Data Release Processing Pipeline

*Simon Krughoff, Phil Marshall and others*


## Introduction

In this Note we describe the sequence of LSST data management (DM) software stack calls needed to emulate the expected Data Release Processing (DRP) pipeline. These recipes were initially developed for the Twinkles project, the pathfinder for the first LSST DESC data challenge, DC1, and were later adapted for the main DC1 simulation. We provide links to the original DC1 recipes at the appropriate points below. 

The target DRP pipeline is described in the LSST Project document [LDM-151](ls.st/ldm-151), _"LSST Data Management Science Pipelines Design."_ We will refer to this document extensively, importing figures and quoting from it as needed. In DC1, the simulated images that we produced were so-called "e-images", ecah one of which emulates a calibrated frame; in DC2, we will produce amplifier (science) and accompanying calibration images, and extend the Twinkles pipeline to include instrument signature removal. We also adopt the recently developed object association code, in order to make `DIAObjects` from our `DIASources`. Other than these two extensions, we expect the DC2 DM pipeline to look similar to the DC1 pipeline. 

<a id='toc'></a>
This Note is organized as follows. We first provide an [overview](#overview) of the DRP pipeline, summarizing the relevant section in [LDM-151](ls.st/ldm-151). Then, we present the recipes in three sections: 
1. [Image Coaddition and Object Detection](#coadds)
2. [Difference Image Analysis and DIASource Detection](#diasources)
3. [DIAObject Generation and Light Curve Forced Photometry](#forcedphot)
We then provide some brief [concluding remarks](#conclusions).  

<a id='overview'></a>
## Data Release Processing Overview


[Back to the table of contents.](#toc)

<a id='coadds'></a>
## Image Coaddition and `Object` Detection

[Back to the table of contents.](#toc)

<a id='diasources'></a>
## Difference Image Analysis and `DIASource` Detection

[Back to the table of contents.](#toc)

<a id='forcedphot'></a>
## `DIAObject` Generation and Light Curve Forced Photometry

[Back to the table of contents.](#toc)

<a id='conclusions'></a>
## Concluding Remarks

[Back to the table of contents.](#toc)

---

## Tables

Tables can be fiddly in `Markdown`. A good place to start is an online table generator like [this one](http://www.tablesgenerator.com/markdown_tables). Then, you'll need some patience. For more on table formatting, see the [`Markdown` cheat-sheet](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables).

|   A   |   B   |      C         |  D  |
|:-----:|:-----:|:--------------:|:---:|
| (deg) | (kpc) | ($M_{\odot}$)  |     |
|  0.4  |  3.4  |  $10^{12.2}$   | R,S |
|  9.6  |  8.2  |  $10^{10.4}$   |  S  |


## Further Resources

For a guide to writing `Markdown` documents, check out this [useful little cheatsheet](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet).